# Self-Driving Car Engineer Nanodegree


## Project: **Behavioral Cloning** 
***

I have decided to use a Jupyter notebook like the previous projects to do the initial work atleast.

---

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, consult the forums for more troubleshooting tips.**  

## Import Packages

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
import cv2
import math
import os
import csv
import random
from scipy import signal
import tensorflow as tf
from glob import glob

from keras.utils import np_utils
from keras.preprocessing import image
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, Lambda
from keras.layers import Cropping2D
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras import optimizers
from keras.optimizers import SGD, Adam, RMSprop

%matplotlib inline

Using TensorFlow backend.


## Read in Images

In [8]:
lines = []

with open('../data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

#print(lines[0])
#print(lines[1])
#print(lines[2:4])
        
images = []
measurements = []

for line in lines[1:]:
    #print(line)
    #print("")
    
    source_path = line[0]
    #print(source_path)
    filename = source_path.split('/')[-1]
    current_path = '../data/IMG/' + filename
    #print(current_path)
    image = cv2.imread(current_path)
    images.append(image)
    measurement = float(line[3])
    measurements.append(measurement)
    
    #image_flipped = np.fliplr(image)
    #images.append(image_flipped)
    #measurement_flipped = -measurement
    #measurements.append(measurement_flipped)

    source_path = line[1]
    #print(source_path)
    filename = source_path.split('/')[-1]
    current_path = '../data/IMG/' + filename
    #print(current_path)
    image = cv2.imread(current_path)
    images.append(image)
    measurement_l = measurement + 0.065
    measurements.append(measurement_l)
    

    source_path = line[2]
    #print(source_path)
    filename = source_path.split('/')[-1]
    current_path = '../data/IMG/' + filename
    #print(current_path)
    image = cv2.imread(current_path)
    images.append(image)
    measurement_r = measurement - 0.065
    measurements.append(measurement_r)



## Setup Training Data

In [9]:
X_train = np.array(images)
y_train = np.array(measurements)

print(len(X_train))
print(len(y_train))

24108
24108


## Nvidia Deep Learning Model

In [10]:
model = Sequential()

model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25), (0,0))))

model.add(Conv2D(24, 5, 5, subsample=(2,2), activation = 'relu'))
model.add(Conv2D(36, 5, 5, subsample=(2,2), activation = 'relu'))
model.add(Conv2D(48, 5, 5, subsample=(2,2), activation = 'relu'))
model.add(Conv2D(64, 3, 3, subsample=(1,1), activation = 'relu'))
model.add(Conv2D(64, 3, 3, subsample=(1,1), activation = 'relu'))

model.add(Flatten())
model.add(Dense(100))
#model.add(Dropout(0.4))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_2 (Lambda)                (None, 160, 320, 3)   0           lambda_input_2[0][0]             
____________________________________________________________________________________________________
cropping2d_2 (Cropping2D)        (None, 65, 320, 3)    0           lambda_2[0][0]                   
____________________________________________________________________________________________________
convolution2d_6 (Convolution2D)  (None, 31, 158, 24)   1824        cropping2d_2[0][0]               
____________________________________________________________________________________________________
convolution2d_7 (Convolution2D)  (None, 14, 77, 36)    21636       convolution2d_6[0][0]            
___________________________________________________________________________________________

## Compile The Model

In [14]:
#adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(optimizer='adam', loss='mse')

## Train The Model

In [15]:
epochs = 4

model.fit(X_train, y_train, validation_split = 0.2, shuffle = True, nb_epoch = epochs)

Train on 19286 samples, validate on 4822 samples
Epoch 1/4
19286/19286 [==============================] - 44s - loss: 0.0101 - val_loss: 0.0104
Epoch 2/4
19286/19286 [==============================] - 44s - loss: 0.0094 - val_loss: 0.0103
Epoch 3/4
19286/19286 [==============================] - 44s - loss: 0.0090 - val_loss: 0.0103
Epoch 4/4
19286/19286 [==============================] - 45s - loss: 0.0087 - val_loss: 0.0105


## Save the Model

In [16]:
model.save('model.h5')